# E-commerce Sales Trend Analysis
)https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data/datawill:
1. Explore sales trends over time
2. Identify top-performing categories
3. Highlight key insights for business decision-making


In [20]:
import pandas as pd

# Load raw dataset
df = pd.read_csv("Amazon Sale Report.csv", low_memory=False)

# 1. Drop completely empty columns (like Unnamed: 2)
df = df.dropna(axis=1, how='all')

# 2. Drop completely empty rows
df = df.dropna(how='all')

# 3. Remove duplicate rows
df = df.drop_duplicates()

# 4. Drop rows with missing critical values (Date, Qty, Amount)
df = df.dropna(subset=['Date', 'Qty', 'Amount'])

# 5. Convert datatypes
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Qty'] = pd.to_numeric(df['Qty'], errors='coerce')
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# 6. Drop rows with invalid values (NaN, negative, or 0)
df = df.dropna(subset=['Date', 'Qty', 'Amount'])
df = df[(df['Qty'] > 0) & (df['Amount'] > 0)]

# 7. Clean column names (remove spaces, lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df = df.drop("unnamed:_22", axis=1)
df


C:\Users\andia\AppData\Local\Temp\ipykernel_24436\2686712762.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


,index,order_id,date,status,fulfilment,sales_channel,ship-service-level,style,sku,category,...,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,b2b,fulfilled-by
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,1,INR,406.0,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,1,INR,329.0,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,1,INR,574.0,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN
5,5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,...,1,INR,824.0,GHAZIABAD,UTTAR PRADESH,201102.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
6,6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,...,1,INR,653.0,CHANDIGARH,CHANDIGARH,160036.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,128970,406-6001380-7673107,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-XL,kurta,...,1,INR,517.0,HYDERABAD,TELANGANA,500013.0,IN,NaN,False,NaN
128971,128971,402-9551604-7544318,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,SET401,SET401-KR-NP-M,Set,...,1,INR,999.0,GURUGRAM,HARYANA,122004.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
128972,128972,407-9547469-3152358,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XXL,Western Dress,...,1,INR,690.0,HYDERABAD,TELANGANA,500049.0,IN,NaN,False,NaN
128973,128973,402-6184140-0545956,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,...,1,INR,1199.0,Halol,Gujarat,389350.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN


In [21]:
# Create SQLite connection (in memory)
conn = sqlite3.connect(":memory:")

# Save df to SQL
df.to_sql("amazon_sales", conn, index=False, if_exists="replace")

113701

In [22]:
query = """
SELECT
    DATE(Date) AS day,
    SUM(Amount) AS total_revenue,
    SUM(Qty) AS total_units
FROM amazon_sales
GROUP BY DATE(Date)
ORDER BY day
"""
trend = pd.read_sql(query, conn)
trend.head()



,day,total_revenue,total_units
0,2022-03-31,98261.0,156
1,2022-04-01,835337.0,1290
2,2022-04-02,882301.0,1377
3,2022-04-03,981822.0,1517
4,2022-04-04,853704.0,1311


In [24]:
query = """
SELECT
    category,
    SUM(qty) AS total_units,
    SUM(amount) AS total_revenue
FROM amazon_sales
GROUP BY category
ORDER BY total_revenue DESC
LIMIT 10
"""
top_categories = pd.read_sql(query, conn)
top_categories.head()

,category,total_units,total_revenue
0,Set,44212,37662424.0
1,kurta,44112,20452141.0
2,Western Dress,13708,10629096.0
3,Top,9725,5203733.0
4,Ethnic Dress,1021,760711.0


In [25]:
query = """
SELECT
    strftime('%Y-%m', Date) AS month,
    SUM(Amount) AS total_revenue,
    SUM(Qty) AS total_units
FROM amazon_sales
GROUP BY strftime('%Y-%m', Date)
ORDER BY month
"""
trend_monthly = pd.read_sql(query, conn)
trend_monthly.head()


,month,total_revenue,total_units
0,2022-03,98261.0,156
1,2022-04,27581851.0,43278
2,2022-05,25119481.0,37222
3,2022-06,22602913.0,33483


In [26]:
query = """
SELECT
    strftime('%Y-%m', Date) AS month,
    Category,
    SUM(Amount) AS revenue
FROM amazon_sales
GROUP BY strftime('%Y-%m', Date), Category
ORDER BY month, revenue DESC
"""
category_trend = pd.read_sql(query, conn)
category_trend.head()


,month,category,revenue
0,2022-03,Set,53884.0
1,2022-03,kurta,33008.0
2,2022-03,Western Dress,5479.0
3,2022-03,Top,4511.0
4,2022-03,Ethnic Dress,1099.0
